# 🚀 Label GCN Classifier

## Approach:
1. **Label Embeddings**: BERT encodes class keywords → 531 label vectors
2. **GCN Refinement**: Propagates info through hierarchy (siblings share info)
3. **Inner Product**: `score = BERT(text) · GCN_refined(label)`

In [1]:
%%time
!pip install -q transformers torch networkx scipy

import os
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torch.nn.functional as F
from transformers import AutoTokenizer, AutoModel
import networkx as nx
from tqdm import tqdm
from collections import Counter, defaultdict
from typing import Dict, List, Tuple
import csv
import gc
import warnings
warnings.filterwarnings('ignore')

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"✓ Device: {device}")

if torch.cuda.is_available():
    torch.cuda.empty_cache()
    gc.collect()
    print(f"  GPU: {torch.cuda.get_device_name(0)}")

✓ Device: cuda
  GPU: NVIDIA L4
CPU times: user 3.18 s, sys: 678 ms, total: 3.86 s
Wall time: 5.06 s


In [2]:
# Config
DATA_DIR = 'data'
OUTPUT_DIR = 'outputs'
NUM_CLASSES = 531
BERT_MODEL = 'bert-base-uncased'
HIDDEN_DIM = 768
GCN_LAYERS = 2
GCN_DROPOUT = 0.3
TOP_K = 20
MIN_SCORE = 0.1

os.makedirs(OUTPUT_DIR, exist_ok=True)
print(f"✓ Config: GCN_LAYERS={GCN_LAYERS}, TOP_K={TOP_K}")

✓ Config: GCN_LAYERS=2, TOP_K=20


In [3]:
# Load functions
def load_corpus(path):
    data = {}
    with open(path, 'r', encoding='utf-8') as f:
        for line in f:
            parts = line.strip().split('\t', 1)
            if len(parts) == 2:
                data[parts[0]] = parts[1]
    return data

def load_classes(path):
    data = {}
    with open(path, 'r', encoding='utf-8') as f:
        for line in f:
            parts = line.strip().split('\t')
            if len(parts) == 2:
                data[int(parts[0])] = parts[1]
    return data

def load_keywords(path):
    data = {}
    with open(path, 'r', encoding='utf-8') as f:
        for line in f:
            parts = line.strip().split(':', 1)
            if len(parts) == 2:
                data[parts[0]] = [kw.strip() for kw in parts[1].split(',')]
    return data

def load_hierarchy(path):
    edges = []
    with open(path, 'r', encoding='utf-8') as f:
        for line in f:
            parts = line.strip().split('\t')
            if len(parts) == 2:
                edges.append((int(parts[0]), int(parts[1])))
    return edges

print("✓ Functions loaded")

✓ Functions loaded


In [4]:
%%time
# Load data
test_corpus = load_corpus(os.path.join(DATA_DIR, 'test/test_corpus.txt'))
id2class = load_classes(os.path.join(DATA_DIR, 'classes.txt'))
class2keywords = load_keywords(os.path.join(DATA_DIR, 'class_related_keywords.txt'))
hierarchy_edges = load_hierarchy(os.path.join(DATA_DIR, 'class_hierarchy.txt'))

print(f"✓ Test: {len(test_corpus):,}")
print(f"✓ Classes: {NUM_CLASSES}")
print(f"✓ Hierarchy edges: {len(hierarchy_edges)}")

✓ Test: 19,658
✓ Classes: 531
✓ Hierarchy edges: 568
CPU times: user 24.4 ms, sys: 9.95 ms, total: 34.4 ms
Wall time: 23 ms


In [5]:
%%time
# Build adjacency matrix
print("Building label graph...")

# Group children by parent
parent2children = defaultdict(list)
for parent, child in hierarchy_edges:
    parent2children[parent].append(child)

# Build graph - connect siblings
G = nx.Graph()
G.add_nodes_from(range(NUM_CLASSES))

for parent, children in parent2children.items():
    for i in range(len(children)):
        for j in range(i+1, len(children)):
            G.add_edge(children[i], children[j])

# Add self-loops
for i in range(NUM_CLASSES):
    G.add_edge(i, i)

# Convert to normalized adjacency
A = nx.adjacency_matrix(G).todense()
A = torch.FloatTensor(A)

# D^{-1/2} * A * D^{-1/2}
D = torch.sum(A, dim=1)
D_inv_sqrt = torch.pow(D, -0.5)
D_inv_sqrt[torch.isinf(D_inv_sqrt)] = 0.0
D_mat = torch.diag(D_inv_sqrt)
A_hat = D_mat @ A @ D_mat
A_hat = A_hat.to(device)

print(f"✓ A_hat: {A_hat.shape}")
print(f"  Edges: {G.number_of_edges()}")

Building label graph...
✓ A_hat: torch.Size([531, 531])
  Edges: 3503
CPU times: user 1.9 s, sys: 256 ms, total: 2.16 s
Wall time: 145 ms


In [6]:
%%time
# Create label embeddings
print("Creating label embeddings...")

tokenizer = AutoTokenizer.from_pretrained(BERT_MODEL)
bert = AutoModel.from_pretrained(BERT_MODEL).to(device)
bert.eval()

label_embeds = []

with torch.no_grad():
    for i in tqdm(range(NUM_CLASSES), desc="Labels"):
        class_name = id2class[i]
        keywords = class2keywords.get(class_name, [class_name.replace('_', ' ')])
        text = ' '.join([kw.replace('_', ' ') for kw in keywords])
        
        enc = tokenizer(text, return_tensors='pt', truncation=True, 
                       max_length=128, padding=True)
        enc = {k: v.to(device) for k, v in enc.items()}
        
        out = bert(**enc)
        emb = out.last_hidden_state.mean(dim=1).squeeze()
        label_embeds.append(emb.cpu())

label_init = torch.stack(label_embeds).to(device)
print(f"✓ Label embeddings: {label_init.shape}")

del bert
torch.cuda.empty_cache()
gc.collect()

Creating label embeddings...


2025-12-19 13:24:08.673211: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1766150648.684716   12272 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1766150648.688345   12272 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-12-19 13:24:08.700419: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: SSE4.1 SSE4.2 AVX AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
Labels: 100%|██████████| 531/531 [00:02<00:00, 181.07it/s]


✓ Label embeddings: torch.Size([531, 768])
CPU times: user 5.04 s, sys: 448 ms, total: 5.49 s
Wall time: 5.54 s


1824

In [7]:
# GCN Model
class LabelGCN(nn.Module):
    def __init__(self, dim, layers=2, dropout=0.3):
        super().__init__()
        self.layers = layers
        self.dropout = dropout
        self.weights = nn.ParameterList([
            nn.Parameter(torch.empty(dim, dim)) for _ in range(layers)
        ])
        for W in self.weights:
            nn.init.xavier_uniform_(W)
    
    def forward(self, H, A):
        for i, W in enumerate(self.weights):
            H = torch.matmul(A, H)
            H = torch.matmul(H, W)
            if i < self.layers - 1:
                H = F.relu(H)
                H = F.dropout(H, p=self.dropout, training=self.training)
        return H

class LabelGCNClassifier(nn.Module):
    def __init__(self, input_dim, label_emb, A, layers=2, dropout=0.3):
        super().__init__()
        self.dropout = dropout
        emb_dim = label_emb.size(1)
        
        self.proj = nn.Linear(input_dim, emb_dim)
        self.gcn = LabelGCN(emb_dim, layers, dropout)
        self.label_emb = nn.Parameter(label_emb.clone())
        self.register_buffer('A', A)
    
    def forward(self, x):
        refined = self.gcn(self.label_emb, self.A)
        x_proj = self.proj(x)
        x_proj = F.dropout(x_proj, p=self.dropout, training=self.training)
        logits = torch.matmul(x_proj, refined.T)
        return logits

print("✓ Models defined")

✓ Models defined


In [8]:
# Initialize model
model = LabelGCNClassifier(
    input_dim=HIDDEN_DIM,
    label_emb=label_init,
    A=A_hat,
    layers=GCN_LAYERS,
    dropout=GCN_DROPOUT
).to(device)

model.eval()

print(f"✓ Model: {sum(p.numel() for p in model.parameters()):,} params")

✓ Model: 2,178,048 params


In [9]:
%%time
# Encode test corpus
print("Encoding test corpus...\n")

tokenizer = AutoTokenizer.from_pretrained(BERT_MODEL)
bert = AutoModel.from_pretrained(BERT_MODEL).to(device)
bert.eval()

test_embs = {}
pids = list(test_corpus.keys())

with torch.no_grad():
    for pid in tqdm(pids, desc="Encoding"):
        text = test_corpus[pid]
        enc = tokenizer(text, return_tensors='pt', truncation=True,
                       max_length=256, padding=True)
        enc = {k: v.to(device) for k, v in enc.items()}
        
        out = bert(**enc)
        emb = out.last_hidden_state.mean(dim=1).squeeze()
        test_embs[pid] = emb.cpu()

print("✓ Encoded")

del bert
torch.cuda.empty_cache()

Encoding test corpus...



Encoding: 100%|██████████| 19658/19658 [01:55<00:00, 170.52it/s]

✓ Encoded
CPU times: user 1min 55s, sys: 524 ms, total: 1min 55s
Wall time: 1min 55s


In [10]:
%%time
# Predict
print("Predicting...\n")

predictions = {}

with torch.no_grad():
    for pid in tqdm(pids, desc="Predict"):
        emb = test_embs[pid].unsqueeze(0).to(device)
        logits = model(emb).squeeze()
        scores = torch.sigmoid(logits).cpu().numpy()
        
        # Top K
        top_k = np.argsort(scores)[::-1][:TOP_K]
        
        # Filter
        cands = [(int(i), scores[i]) for i in top_k if scores[i] >= MIN_SCORE]
        
        if len(cands) < 2:
            cands = [(int(i), scores[i]) for i in top_k[:3]]
        
        predictions[pid] = [c[0] for c in cands[:3]]

print("✓ Done")

Predicting...



Predict: 100%|██████████| 19658/19658 [00:07<00:00, 2675.83it/s]

✓ Done
CPU times: user 7.34 s, sys: 28.1 ms, total: 7.37 s
Wall time: 7.35 s


In [11]:
# Analysis
all_classes = []
for labels in predictions.values():
    all_classes.extend(labels)
counts = Counter(all_classes)

print(f"\n{'='*60}")
print(f"RESULTS")
print(f"{'='*60}")
print(f"Unique classes: {len(counts)}/531")

if len(counts) >= 250:
    print(f"Expected: 0.40-0.55 ✓")
elif len(counts) >= 150:
    print(f"Expected: 0.32-0.45")
else:
    print(f"Expected: 0.25-0.35")

vals = list(counts.values())
print(f"\nMin: {min(vals)}, Max: {max(vals)}, Mean: {np.mean(vals):.1f}")

print(f"\nTop 10:")
for cid, cnt in counts.most_common(10):
    print(f"  {id2class[cid][:40]:40s}: {cnt:4d} ({cnt/len(predictions)*100:.1f}%)")
print(f"{'='*60}")


RESULTS
Unique classes: 161/531
Expected: 0.32-0.45

Min: 1, Max: 15436, Mean: 366.3

Top 10:
  toys_games                              : 15436 (78.5%)
  baby_products                           : 5828 (29.6%)
  hair_perms_texturizers                  : 1667 (8.5%)
  styling_tools                           : 1615 (8.2%)
  styling_products                        : 1615 (8.2%)
  hair_loss_products                      : 1591 (8.1%)
  shampoos                                : 1574 (8.0%)
  hair_color                              : 1530 (7.8%)
  hair_relaxers                           : 1522 (7.7%)
  hair_scalp_treatments                   : 1500 (7.6%)


In [12]:
# Save
out = os.path.join(OUTPUT_DIR, 'label_gcn_predictions.csv')

with open(out, 'w', newline='', encoding='utf-8') as f:
    writer = csv.writer(f)
    writer.writerow(['id', 'labels'])
    for pid in sorted(predictions.keys(), key=lambda x: int(x)):
        writer.writerow([pid, ','.join(map(str, predictions[pid]))])

print(f"\n✓ Saved: {out}")

df = pd.read_csv(out)
print("\nSample:")
print(df.head(10))

print(f"\n{'='*60}")
print(f"LABEL GCN COMPLETE")
print(f"{'='*60}")
print(f"Unique: {len(counts)}/531")
print(f"\n📤 Submit to Kaggle")
print(f"{'='*60}")


✓ Saved: outputs/label_gcn_predictions.csv

Sample:
   id      labels
0   0    3,80,461
1   1    3,90,473
2   2     3,40,46
3   3    40,3,205
4   4     3,40,90
5   5  40,169,442
6   6    3,90,338
7   7   3,237,143
8   8   3,461,335
9   9   3,116,143

LABEL GCN COMPLETE
Unique: 161/531

📤 Submit to Kaggle
